In [ ]:
! pip install --upgrade --quiet  langchain-openai tavily-python

In [ ]:
! pip install --upgrade langchain

In [1]:
import os
import sys
import numpy as np
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, CommaSeparatedListOutputParser, JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from dotenv import load_dotenv
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import MessagesPlaceholder
from langchain_community.document_loaders import WebBaseLoader
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores.faiss import FAISS
from langchain.chains import create_retrieval_chain
from langchain.agents import AgentExecutor, create_openai_functions_agent
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.tools.retriever import create_retriever_tool

current_path = os.getcwd()
parrent_path = os.path.abspath(os.path.join(current_path, '..'))
sys.path.append(parrent_path)

from Update_Git import git_add, git_commit, git_push

file_path = os.path.join(current_path, 'Test_6.ipynb')
git_add(file_path)
git_commit('Updated test 6')
git_push('main')

os.environ.pop("OPENAI_API_KEY", None) # Because it loads a key from some place I dont know!
env_path = os.path.join(current_path, ".env")
load_dotenv(dotenv_path=env_path)

/Users/taha/Documents/Python_codes/Dev-playground/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3672: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
USER_AGENT environment variable not set, consider setting it to identify your requests.


True

In [2]:

#___________________-   Chatting with GPT-3.5   -______________________#

# Get the context from the web
loader = WebBaseLoader("https://en.wikipedia.org/wiki/FreeSurfer")  # Load the document from the web
docs = loader.load()  # Extract the content

# Transform the document into a format that can be used by the retriever
splitter = RecursiveCharacterTextSplitter(
    chunk_size=400,  # Each chunk has a max size of 200 characters
    chunk_overlap=50  # Overlapping of 50 characters between chunks for context continuity
)
split_docs = splitter.split_documents(docs)  # Split the documents

embedding = OpenAIEmbeddings()  # Generate embeddings using OpenAI
vector_store = FAISS.from_documents(split_docs, embedding=embedding)  # Store documents in FAISS index - Facebook AI Similarity Search

retriever = vector_store.as_retriever(search_kwaargs = {"k": 5})  # Convert vector store into a retriever


model = ChatOpenAI(
    model = 'gpt-3.5-turbo',
    temperature=0.5
)

prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a friendly assisstant called gyris'),
    MessagesPlaceholder(variable_name = "chat_history"),
    ('human', '{input}'),
    MessagesPlaceholder(variable_name = "agent_scratchpad")
])

# tool for our document 
retriever_tool = create_retriever_tool(
    retriever,
    "toolbox_search", # identifier for the tool
    "Use this tool when searching for information about FreeSurfer" # description of the tool
    )

# tool for our web search
search = TavilySearchResults() 

# List of tools - we have 2 tools here
tools = [search, retriever_tool]

agent = create_openai_functions_agent(
    llm = model,
    prompt = prompt,
    tools = tools
)

agentExecutor = AgentExecutor(
    agent = agent,
    tools = tools
)

def process_chat(agentExecutor, user_input, chat_history):

    response = agentExecutor.invoke({
    "input": user_input,
    "chat_history": chat_history
    })

    return response["output"]


if __name__ == "__main__":
    chat_history = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            break
        response = process_chat(agentExecutor, user_input, chat_history)
        chat_history.append(HumanMessage(content=user_input))
        chat_history.append(AIMessage(content=response))
        print("AI: ", response)


/var/folders/t4/9lhz3gg574q401jn3rjb5zj00000gn/T/ipykernel_5307/2424085529.py:40: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search = TavilySearchResults()


ValidationError: 1 validation error for TavilySearchAPIWrapper
  Value error, Did not find tavily_api_key, please add an environment variable `TAVILY_API_KEY` which contains it, or pass `tavily_api_key` as a named parameter. [type=value_error, input_value={}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.11/v/value_error